# House prices in King County Washington

We'll start the class with PowerPoint slides covering a definition of what it means for an algorithm to "learn from data".  (The slides are available <a href="../DefLearning.pdf">here</a>.)

* Can we "learn from data" using decision tree regression on the [King County dataset](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)?  (Here are the [column definitions](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction/discussion/207885) for this dataset.)

## Using a restricted decision tree regressor

In [1]:
import pandas as pd
import numpy as np
import altair as alt

ModuleNotFoundError: No module named 'altair'

In [ ]:
%pip install altair

In [2]:
df = pd.read_csv("kc_house_data.csv")

In [3]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

We need our features for decision trees to be numerical.  All of the columns in this datasest are already numerical except for the "date" column.  We will just ignore that one (and the "id" column).

In [4]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

Here is an easy way to get all the columns, starting with the column at index 3.

In [5]:
cols = df.columns[3:]
cols

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

To see if we can "learn from data" in this case, we will measure our performance on a test set.

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df[cols],
    df['price'],
    random_state=0,
    train_size=0.8
)

This is a large dataset; our training set has 17290 instances.

In [8]:
X_train.shape

(17290, 18)

We will model the price of a house using a decision tree.

In [9]:
from sklearn.tree import DecisionTreeRegressor

Creating the `DecisionTreeRegressor` object is similar to creating a `DecisionTreeClassifier` object, but here we specify to use Mean Absolute Error as our metric; that is what the `criterion="absolute_error"` keyword argument is doing.  You can read about the different options, as well as the default being "squared_error", in the [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html).

One motivation for using Mean Absolute Error is that if we are off by (for example) 2 million dollars on a house price prediction, we don't want that error getting squared, which would (in my estimation) over-emphasize these expensive houses.

In [10]:
reg = DecisionTreeRegressor(max_depth=15, max_leaf_nodes=20, criterion="absolute_error")

Mean absolute error is closely connected to median, so that is what we will use for our baseline prediction.  (Exercise: show that if you have a list of numbers, the estimate which minimizes Mean Absolute Error is the median, and the estimate which minimizes the Mean Squared Error is the mean.)

In [11]:
df["price"].median()

450000.0

Let's start out measuring the performance of our baseline model, which always predicts the median value.  There are many ways to make a list (or array, etc) of the same number.  We will use a method that is reminiscent of the `ones` function from Matlab.

In [12]:
df.price.median()*np.ones(4)

array([450000., 450000., 450000., 450000.])

In [13]:
from sklearn.metrics import mean_absolute_error

Here is the performance of our baseline algorithm.

In [14]:
# baseline
mean_absolute_error(df.price.median()*np.ones(len(y_test)), y_test)

215282.15660421003

If we get a smaller (better) mean absolute error when using our decision tree regressor, then we can say that our algorithm "learned from data".

In [15]:
reg.fit(X_train, y_train)

DecisionTreeRegressor(criterion='absolute_error', max_depth=15,
                      max_leaf_nodes=20)

Because the following value is lower than the baseline (significantly lower), we have "learned from data" in this case.

In [16]:
# decision tree
mean_absolute_error(reg.predict(X_test), y_test)

109940.3937080731

As with the decision tree classifiers, we can visualize the decision tree.  (I wouldn't try making these visualizations with a much larger tree.)  One difference is that in this case, the Mean Absolute Error is reported.  (If we had used a different metric, that other metric would be reported.)  Another difference is that the outputs are numbers, not classes (and the shading colors represent the size of those numbers).  The same number is output on each leaf (the median of the prices for the corresponding region).

In [17]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [18]:
fig = plt.figure(figsize=(200,100))
_ = plot_tree(
    reg,
    feature_names=reg.feature_names_in_,
    filled=True
)

## An unrestricted decision tree

Here we briefly try out a decision tree with no restrictions (it can have as large of depth and as many leaf nodes as it wants).  We should expect there will be overfitting in this case, although that's not our focus here.

In [19]:
reg2 = DecisionTreeRegressor(criterion="absolute_error")

In [20]:
reg2.fit(X_train, y_train)

DecisionTreeRegressor(criterion='absolute_error')

What depth does this unrestricted tree have, and how many leaves?

In [21]:
reg2.get_depth()

44

You should interpret the next result as saying that this decision tree is subdividing the feature space into 16440 regions.

In [22]:
reg2.get_n_leaves()

16444

If we just evaluate `feature_importances_`, it is difficult to interpret; which numbers belong to which columns?

In [23]:
reg2.feature_importances_

array([0.00946411, 0.01581117, 0.17327192, 0.03741738, 0.00481756,
       0.01366853, 0.01177944, 0.00923557, 0.19536418, 0.04083494,
       0.01100631, 0.04526546, 0.00379778, 0.02297101, 0.24533171,
       0.07674623, 0.05153238, 0.03168431])

In [24]:
reg2.feature_names_in_

array(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], dtype=object)

Alternatively, we could evaluate `cols`, but the `feature_names_in_` procedure is more reliable.

In [25]:
cols

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

If all we care about is the correspondence between numbers and feature names, it would be more natural to put this information into a pandas Series.

In [26]:
s = pd.Series(reg2.feature_importances_, index=reg2.feature_names_in_)
s

bedrooms         0.009464
bathrooms        0.015811
sqft_living      0.173272
sqft_lot         0.037417
floors           0.004818
waterfront       0.013669
view             0.011779
condition        0.009236
grade            0.195364
sqft_above       0.040835
sqft_basement    0.011006
yr_built         0.045265
yr_renovated     0.003798
zipcode          0.022971
lat              0.245332
long             0.076746
sqft_living15    0.051532
sqft_lot15       0.031684
dtype: float64

But eventually we want to plot this information using Altair, so putting this data into two distinct columns in a pandas DataFrame is more useful.

In [27]:
s.to_frame().reset_index()

,index,0
0,bedrooms,0.009464
1,bathrooms,0.015811
2,sqft_living,0.173272
3,sqft_lot,0.037417
4,floors,0.004818
5,waterfront,0.013669
6,view,0.011779
7,condition,0.009236
8,grade,0.195364
9,sqft_above,0.040835


Another way is to make the DataFrame directly (without first making the pandas Series).

In [28]:
df_feat = pd.DataFrame({"importance": reg2.feature_importances_, "feature": reg2.feature_names_in_})

In [29]:
df_feat

,importance,feature
0,0.009464,bedrooms
1,0.015811,bathrooms
2,0.173272,sqft_living
3,0.037417,sqft_lot
4,0.004818,floors
5,0.013669,waterfront
6,0.011779,view
7,0.009236,condition
8,0.195364,grade
9,0.040835,sqft_above


Using a bar chart to visualize this information is very natural.  We will use horizontal bars just for something different.

In [30]:
alt.Chart(df_feat).mark_bar().encode(
    x="importance",
    y="feature"
)

NameError: name 'alt' is not defined

The reason I wanted to make this chart using an unrestricted tree is so that more (in fact all) of the columns would be non-zero.  I didn't say this during lecture, but because there is overfitting (almost surely), these feature importances might not be as meaningful as if we had a somewhat less flexible decision tree.